# Analysis for case study 3

In [2]:
import pickle
import numpy as np
import pandas as pd

## Load in data

In [3]:
# load data
ds = ['carla']
data = []
for d in ds:      
    tab_file = 'study-3-collaborative-{}-seq-res-EXP2.p'.format(d)
    with open(tab_file, 'rb') as f:
        data.append(pickle.load(f))
        
# convert to dataframe
df = pd.concat(pd.DataFrame.from_dict(dat) for dat in data)
df.head()

,Case,Dataset,Trial,Metrics_perception_object_3d_tot_TP,Metrics_perception_object_3d_tot_FP,Metrics_perception_object_3d_tot_FN,Metrics_perception_object_3d_tot_T,Metrics_perception_object_3d_mean_precision,Metrics_perception_object_3d_mean_recall,Metrics_tracking_HOTA_HOTA,...,Metrics_tracking_Count_Frames,Metrics_prediction_agg_ADE,Metrics_prediction_agg_FDE,Metrics_prediction_std_ADE,Metrics_prediction_std_FDE,Metrics_prediction_n_with_truth,Metrics_prediction_n_objects,Metrics_collaborative_n_collab_detections_median,Metrics_collaborative_n_collab_sensors_in_view_median,Metrics_collaborative_n_collab_sensors_total_median
0,1,CARLA,0,409,0,151,560,1.0,0.736389,"[0.7988125108763833, 0.7988125108763833, 0.798...",...,50,4.051588,8.995710,0.912676,2.052923,1073,1073,54.0,5.0,39.0
1,2,CARLA,0,409,0,151,560,1.0,0.736389,"[0.772913900803881, 0.772913900803881, 0.77291...",...,50,4.197255,9.137468,0.860767,1.805356,1120,1120,54.0,5.0,39.0
2,3,CARLA,0,409,0,151,560,1.0,0.736389,"[0.7882661881870536, 0.7882661881870536, 0.788...",...,50,4.377758,9.559689,1.056035,1.709084,1002,1002,24.5,5.0,39.0
3,4,CARLA,0,409,0,151,560,1.0,0.736389,"[0.7410991363687861, 0.7410991363687861, 0.741...",...,50,3.049756,5.839894,2.174768,5.004019,371,371,NaN,NaN,NaN


## Extract Data of Interest

Rows: case numbers

Columns: metrics

Within columns: each dataset's mean and std values across trials

In [4]:
metrics_of_interest = {
                            'Metrics_collaborative_n_collab_sensors_in_view_median':'Collab: \#S-in-range/frame',
                            'Metrics_collaborative_n_collab_detections_median':'Collab: Dets/frame',
                            'Metrics_tracking_HOTA_HOTA':'Trk: HOTA',
                            'Metrics_tracking_CLEAR_MOTA':'Trk: MOTA',
                            'Metrics_tracking_CLEAR_MOTP':'Trk: MOTP',
                            'Metrics_prediction_std_ADE':'Pred: ADE',
                            'Metrics_prediction_std_FDE':'Pred: FDE'}

expansion_types = {
                    'Metrics_collaborative_n_collab_sensors_in_view_median':None,
                    'Metrics_collaborative_n_collab_detections_median':None,
                    'Metrics_tracking_HOTA_HOTA':'value-at-middle',
                    'Metrics_tracking_CLEAR_MOTA':'value-at-middle',
                    'Metrics_tracking_CLEAR_MOTP':'value-at-middle',
                    'Metrics_prediction_std_ADE':None,
                    'Metrics_prediction_std_FDE':None}

In [5]:
# Convert to table format in triple slash format
mark_best_in_cell = False
mask_best_in_col = False

# subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'
subrow_formatter = '{}'

dses = df["Dataset"].unique()
print(f'Triple slash dataset ordering is: {dses}')
res_triple_agg = []
for i_case in df['Case'].unique():
    res_triple = {'Case':i_case}
    res_triple.update({'Data':subrow_formatter.format(*[d[0].upper() for d in dses])})
    for met_k, met_v in metrics_of_interest.items():
        res_met_triple = []
        for dataset in df['Dataset'].unique():
            met_res = df[(df['Case'] == i_case) & (df['Dataset'] == dataset)][met_k]
            if expansion_types[met_k] == 'max-over-dict':
                met_vals = [np.max(met.values()) for met in met_res]
            elif expansion_types[met_k] == 'max-over-list':
                met_vals = [np.max(met) for met in met_res]
            elif expansion_types[met_k] == 'value-at-middle':
                met_vals = [np.median(met) for met in met_res]  # median accomplishes middle value
            elif expansion_types[met_k] is None:
                met_vals = met_res
            else:
                raise NotImplementedError('Expansion {} not implemented'.format(expansion_types[met_k]))
            mn, std = np.nanmedian(met_vals), np.nanstd(met_vals)
            res_met_triple.append((mn, std))
            
        # triple slash format -- underlining best in cell
        if mark_best_in_cell and (not all([np.isnan(mn) for mn, _ in res_met_triple])):
            best_idx = metric_best_evaluator[met_k]([mn for mn, _ in res_met_triple])
        else:
            best_idx = None
        res_met_triple_new = []
        for i, (mn, std) in enumerate(res_met_triple):
            if np.isnan(mn):
                wstr = 'N/A'
            else:
                if (best_idx is not None) and (i==best_idx):
                    wstr = f'\\underline{{{mn:4.2f} +/- {std:4.2f}}}'
                else:
                    wstr = f'{mn:4.2f} +/- {std:4.2f}'
            res_met_triple_new.append(wstr)

        # Format the whole triple
        res_trip = subrow_formatter.format(*res_met_triple_new)
        res_triple.update({met_v:res_trip})
    res_triple_agg.append(res_triple)

Triple slash dataset ordering is: ['CARLA']


/home/spencer/.cache/pypoetry/virtualenvs/avstack-studies-CCMTMq_C-py3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/spencer/.cache/pypoetry/virtualenvs/avstack-studies-CCMTMq_C-py3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [6]:
pd.set_option('display.max_colwidth',1000)
df_triple = pd.DataFrame(res_triple_agg)
lat_str = df_triple.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
print(lat_str)

\begin{tabular}{rllllllll}
\toprule
 Case & Data & Collab: \#S-in-range/frame & Collab: Dets/frame &     Trk: HOTA &     Trk: MOTA &     Trk: MOTP &     Pred: ADE &     Pred: FDE \\ \midrule
\midrule
    1 &    C &              5.00 +/- 0.00 &     54.00 +/- 0.00 & 0.80 +/- 0.00 & 0.62 +/- 0.00 & 3.12 +/- 0.00 & 0.91 +/- 0.00 & 2.05 +/- 0.00 \\ \midrule
    2 &    C &              5.00 +/- 0.00 &     54.00 +/- 0.00 & 0.77 +/- 0.00 & 0.61 +/- 0.00 & 2.98 +/- 0.00 & 0.86 +/- 0.00 & 1.81 +/- 0.00 \\ \midrule
    3 &    C &              5.00 +/- 0.00 &     24.50 +/- 0.00 & 0.79 +/- 0.00 & 0.59 +/- 0.00 & 2.96 +/- 0.00 & 1.06 +/- 0.00 & 1.71 +/- 0.00 \\ \midrule
    4 &    C &                        N/A &                N/A & 0.74 +/- 0.00 & 0.61 +/- 0.00 & 3.13 +/- 0.00 & 2.17 +/- 0.00 & 5.00 +/- 0.00 \\ \midrule
\bottomrule
\end{tabular}



/tmp/ipykernel_1426006/1648327297.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  lat_str = df_triple.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
